# Iris Case Study: Encoding string target variables

## Load Libraries

In [2]:
from pandas import read_csv
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

## Load Data

In [4]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
data = read_csv(url, header = None)

## Split data into X/Y

In [5]:
X = data.values[:,0:4]
Y = data.values[:,4]

## Encode String Class (target variable) as integers

In [6]:
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(Y)
label_encoded_y = label_encoder.transform(Y)

## Split into train/test sets

In [7]:
seed = 7
test_size = 0.33
X_train, X_test, Y_train, Y_test = train_test_split(X, label_encoded_y,
                                                    test_size = 0.33, 
                                                    random_state=seed)

## Fit Model on Training Set

In [8]:
model = XGBClassifier()
model.fit(X_train, Y_train)
print(model)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)


## Make predictions and evaluate

In [12]:
#Predict
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
#Evaluate
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 92.00%


# Breast Cancer Case Study: categorical datasets
Some datasets only contain categorical data, for example the breast cancer dataset. This dataset describes the technical details of breast cancer biopsies and the prediction task is to predict whether or not the patient has a recurrence of cancer, or not.

## Load Libraries

In [35]:
import numpy
from pandas import read_csv
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

## Load Data

In [26]:
filename = 'breastcancer.csv'
data = read_csv(filename, header = None, delimiter=",", skiprows = 105, nrows = 286)
data.shape

(286, 10)

In [28]:
data.head(5)

,0,1,2,3,4,5,6,7,8,9
0,'40-49','premeno','15-19','0-2','yes','3','right','left_up','no','recurrence-events'
1,'50-59','ge40','15-19','0-2','no','1','right','central','no','no-recurrence-events'
2,'50-59','ge40','35-39','0-2','no','2','left','left_low','no','recurrence-events'
3,'40-49','premeno','35-39','0-2','yes','3','right','left_low','yes','no-recurrence-events'
4,'40-49','premeno','30-34','3-5','yes','2','left','right_up','no','recurrence-events'


In [51]:
data.dtypes

0    object
1    object
2    object
3    object
4    object
5    object
6    object
7    object
8    object
9    object
dtype: object

All 9 input variables are categorical and described in string format. The problem is a binary classification prediction problem and the output class values are also described in string format. They need to be encoded.

XGBoost may assume that encoded integer values for each input variable have an ordinal relationship. For example that left-up encoded as 0 and left-low encoded as 1 for the breast-quad variable have a meaningful relationship as integers. In this case, this assumption is untrue. Instead, we must map these integer values onto new binary variables, one new variable for each categorical value.

This can be modeled as 5 binary variables in a process called ONE HOT ENCODING using the OneHotEncoder class in sklearn. 

## Encode variables

In [58]:
from sklearn.preprocessing import OneHotEncoder
from numpy import column_stack

#Split input/target
X = data.values[:,1:9]
X = X.astype(str)
Y = data.values[:,9]

#Encode string input variables
columns = []
for i in range(0, X.shape[1]):
    label_encoder = LabelEncoder()
    feature = label_encoder.fit_transform(X[:,i])
    feature = feature.reshape(X.shape[0], 1)
    onehot_encoder = OneHotEncoder(sparse=False)
    feature = onehot_encoder.fit_transform(feature)
    columns.append(feature)

#Collapse columns into array
encoded_x = column_stack(columns)
print("X shape: : ", encoded_x.shape)

#Encode string target variable
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(Y)
encoded_y = label_encoder.transform(Y)

X shape: :  (286, 37)


## Split into Train/Test sets; fit model, predict and evaluate

In [62]:
# Split Train/Test
seed = 7
test_size = 0.33
X_train, X_test, Y_train, Y_test = train_test_split(encoded_x, encoded_y, 
                                                   test_size = test_size, random_state = seed)

# Fit Model
model = XGBClassifier()
model.fit(X_train, Y_train)
print(model)

# Predict and Evaluate
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)
Accuracy: 71.58%
